### **Распаковка данных**

In [ ]:
!wget https://onti2020.ai-academy.ru/task/rucos.zip

--2021-03-04 08:13:06--  https://onti2020.ai-academy.ru/task/rucos.zip
Resolving onti2020.ai-academy.ru (onti2020.ai-academy.ru)... 213.159.215.214
Connecting to onti2020.ai-academy.ru (onti2020.ai-academy.ru)|213.159.215.214|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56208690 (54M) [application/zip]
Saving to: ‘rucos.zip’

rucos.zip           100%[===================>]  53.60M  14.2MB/s    in 4.6s    

2021-03-04 08:13:12 (11.7 MB/s) - ‘rucos.zip’ saved [56208690/56208690]



In [ ]:
!unzip rucos.zip

Archive:  rucos.zip
   creating: RuCoS/
  inflating: __MACOSX/._RuCoS        
  inflating: RuCoS/.DS_Store         
  inflating: __MACOSX/RuCoS/._.DS_Store  
  inflating: RuCoS/rucos_test.jsonl  
  inflating: __MACOSX/RuCoS/._rucos_test.jsonl  
  inflating: RuCoS/rucos_val.jsonl   
  inflating: __MACOSX/RuCoS/._rucos_val.jsonl  
  inflating: RuCoS/rucos_train.jsonl  
  inflating: __MACOSX/RuCoS/._rucos_train.jsonl  


### **Загрузка библиотек**

In [ ]:
!pip install transformers[sentencepiece]
!pip install datasets
!pip install jsonlines

     |████████████████████████████████| 1.9MB 9.7MB/s 
     |████████████████████████████████| 3.2MB 54.1MB/s 
     |████████████████████████████████| 890kB 74.6MB/s 
     |████████████████████████████████| 1.1MB 63.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=feb776637902426b6b3f52a18f300ef3e514720cdf14a4bb9b8c843a47708d10
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 194kB 8.3MB/s 
     |████████████████████████████████| 112kB 12.1MB/s 
     |████████████████████████████████| 245kB 9.2MB/s 


### **Импорт**

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator
from datasets import load_dataset, load_metric, Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import json
import datasets

### **Загрузка train, val, test**

In [ ]:
import json

with open('/content/RuCoS/rucos_train.jsonl', 'r') as json_file:
      train = list(json_file)


with open('/content/RuCoS/rucos_val.jsonl', 'r') as json_file:
      validate = list(json_file)


with open('/content/RuCoS/rucos_test.jsonl', 'r') as json_file:
      test = list(json_file)

### **Внешний вид данных**

In [ ]:
eval(train[0])

{'idx': 0,
 'passage': {'entities': [{'end': 57, 'start': 47},
   {'end': 209, 'start': 199},
   {'end': 242, 'start': 223},
   {'end': 291, 'start': 281},
   {'end': 377, 'start': 371},
   {'end': 592, 'start': 582},
   {'end': 615, 'start': 607},
   {'end': 631, 'start': 617},
   {'end': 640, 'start': 633},
   {'end': 660, 'start': 654},
   {'end': 717, 'start': 709},
   {'end': 768, 'start': 758},
   {'end': 798, 'start': 789},
   {'end': 829, 'start': 819}],
  'text': 'Наблюдатели полагают, что подоплекой теракта в Домодедово является провал кавказской политики российского правительства, указывает немецкая печать. Немецкая печать продолжает комментировать теракт в Домодедово. Так, газета Süddeutsche Zeitung пишет:\nНу, конечно же, после взрыва в Домодедово вновь обнаруживается "кавказский след". Его обнаруживают почти всегда, когда в России взрывается бомба в метро, в поезде или на рынке. И неважно, что очевидцы нередко не в состоянии сказать, был ли преступник женщиной в чадре или

### **Предобработка**

Берем весь 'text' из train записей. 

Сохраняем уникальные символы

In [ ]:
text = ''
for i in train:
  text += eval(i)['passage']['text']
unique_t = set(text)

Отрезаем 255 символов в начале unique_t, т.к. все символы после 255 индекса либо встречаются редко, либо же являются эмоджи. Результат сохраним в problem_symb. Также мы добавляем к листу еще один лист с отрезанными уникальными значениями, но из validation.jsonl passage text.

In [ ]:
problem_symb = sorted(list(unique_t))[255:] + ['\u200b','\u200d','‐','‑','–','—','’','“','”','…','\u202f','€','₽','№','−','・','拉','法','维','韦','🏠','🏫','🏽','👏','👩','🚕']

Перемешаем выборку 

(Блок кода ниже используется для случайного отобора некоторого количества записей. В данный момент отбираются все)

In [ ]:
import random
train = random.sample(train,len(train))
validate = random.sample(validate,len(validate))

Функции для предобработки текста

In [ ]:
def find_body(contexts):
  """
  Для нахождения основного текста и удаления highlight
  contexts - это passage text, к которому применили split("@highlight")
  context - основной текст
  """
  c = -10
  context = 0
  for i in contexts:
    if len(i) > c:
      c = len(i)
      context = i
  return context

def find_title(train_ex):
  """
  Для проверки нахождения 'source' в элементе train
  """
  if 'source' in train_ex:
    return train_ex['source']
  return ''

def FUNC_TEST(text):
  """
  Замена отобранных блоками выше символов на пустую строку.
  (Правильнее было бы заменить некоторые символы по смыслу,
  но решение на лидерборде было выполнено так)
  """
  for i in problem_symb:
    text = text.replace(i,' ')
  return text.replace('\n',' ').replace('\t',' ')

In [ ]:
# Отбираем первую запись train. Получаем все поля и значения записи
ans = eval(train[0])['qas'][0]['answers'][0]
context = eval(train[0])['passage']['text']
idx = eval(train[0])['idx']
query = eval(train[0])['qas'][0]['query']
source = find_title(eval(train[0]))
start_ind, end_ind = ans['start'],ans['end']
ans_text = context[start_ind:end_ind]

#Создаем словарь, который будем заполнять значениями train
train_arr = {
            'answer': [{'answer_start': [start_ind],'text':[ans_text]}],
            'context':[context],
            'id':[idx],
            'question': [query],
            'title' : [source]
              }

# Запускаем цикл извлечения признаков и добавления их в словарь.
# К train добавляем практически всю валидационную выборку, за исключением 2тыс. записей
# Весь датасет приводим к формату датасета SQUaD

for i in train + validate[:-2000]: 
  ans =   eval(i)['qas'][0]['answers']      
  contexts = eval(i)['passage']['text'].split('@highlight')
  context =  FUNC_TEST(find_body(contexts)) 
  idx = eval(i)['idx']
  query = FUNC_TEST(eval(i)['qas'][0]['query'])                   
  source = FUNC_TEST(find_title(eval(i))) 
  ans_list = []
  context = context + source 
  c = -10
  for a in ans:
    # Находим самый длинный ответ из предложенных
    if len(FUNC_TEST(a['text'])) > c:
      c = len(FUNC_TEST(a['text']))
      ans_list.append([a['text'],a['start']])
  if c < 2: # Если самый длинный ответ по длине меньше 2, то пропускаем запись
    continue

  # Соединяем запись на данной итерацией с предыдущей
  two = {
                'answer': [{'answer_start': [ans_list[-1][1]],'text':[ans_list[-1][0]]}],
                'context':[context],
                'id':[idx],
                'question': [query],
                'title' : [source]
                  }
  train_arr = {key: value + two[key] for key, value in train_arr.items()}

# Приводим наш словарь к типу Dataset
train_dataset = Dataset.from_dict(train_arr)


# Тоже самое и для validate

ans = eval(validate[-2])['qas'][0]['answers'][0]
context = eval(validate[-2])['passage']['text']
idx = eval(validate[-2])['idx']
query = eval(validate[-2])['qas'][0]['query']
source = find_title(eval(validate[-2]))
start_ind, end_ind = ans['start'],ans['end']
ans_text = context[start_ind:end_ind]



valid_arr = {
            'answer': [{'answer_start': [start_ind],'text':[ans_text]}],
            'context':[context],
            'id':[idx],
            'question': [query],
            'title' : [source]
              }


for i in validate[-1:]:

  ans =   eval(i)['qas'][0]['answers']  
  contexts = eval(i)['passage']['text'].split('@highlight')
  context =  FUNC_TEST(find_body(contexts))
  idx = eval(i)['idx']
  query = FUNC_TEST(eval(i)['qas'][0]['query']) 
  source = FUNC_TEST(find_title(eval(i)))
  ans_list = []
  context = context + source 

  ans_list = []
  c = -10
  for a in ans:
    if len(FUNC_TEST(a['text'])) > c:
      c = len(FUNC_TEST(a['text']))
      ans_list.append([a['text'],a['start']])

  if c<2:
    continue

  two = {
                'answer': [{'answer_start': [ans_list[-1][1]],'text':[ans_list[-1][0]]}],
                'context':[context],
                'id':[idx],
                'question': [query],
                'title' : [source]
                  }
  valid_arr = {key: value + two[key] for key, value in valid_arr.items()}

validate_dataset = Dataset.from_dict(valid_arr)

In [ ]:
# Приводим test к виду SQUaD, пропуская ответ

ans = ''
context = eval(test[0])['passage']['text']
idx = eval(test[0])['idx']
query = eval(test[0])['qas'][0]['query']
source = find_title(eval(test[0]))
start_ind, end_ind = 0,0
ans_text = ''


test_arr = {
            'answer': [''],
            'context':[context],
            'id':[idx],
            'question': [query],
            'title' : [source]
              }


for i in test:
  contexts = eval(i)['passage']['text'].split('@highlight')
  context =  FUNC_TEST(find_body(contexts)) 
  idx = eval(i)['idx']
  query = FUNC_TEST(eval(i)['qas'][0]['query'])                   
  source = FUNC_TEST(find_title(eval(i)))
  ans_list = []
  context = context + source 

  ans_list = []

  two = {
                'answer': [''],
                'context':[context],
                'id':[idx],
                'question': [query],
                'title' : [source]
                  }
  test_arr = {key: value + two[key] for key, value in test_arr.items()}

test_dataset = Dataset.from_dict(test_arr)

In [ ]:
# Приводим тип Dataset к DatasetDict
dataset_train = DatasetDict({'train':train_dataset,'validate':validate_dataset})
dataset_test = DatasetDict({'test':test_dataset})

In [ ]:
# Параметры обучения

model_checkpoint = 'deepset/xlm-roberta-large-squad2'
batch_size = 4
learning_rate = 2.513700166451428e-06 # ИЛИ 1e-5
weight_decay = 0.006834443975239724 # ИЛИ 0.01
lr_schedule = 'LinearWarmup'
warmup_proportion = 0.2
doc_stride=128
max_query_length=64
max_length = 384

### **Токенизация**

In [ ]:
# Загружаем токенизатор
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
# Находим пример токенизации вопроса и ответа по длине input_ids 
for i, example in enumerate(dataset_train["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = dataset_train["train"][i]

In [ ]:
# Пример токенизации с предыдущего блока сохраняем в переменную
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)

In [ ]:
# ID токена примера с индексами [0][1] и смещение [0][1]
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]

In [ ]:
answers = example["answer"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])
sequence_ids = tokenized_example.sequence_ids()
# Начальный индекс токена текущего промежутка в тексте
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# Последний индекс токена текущего промежутка в тексте
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Если ответ выходит за допустимый диапазон, то он помечается индексом CLS
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Token_start_index и token_end_index на два конца ответа.
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("Ответа нет")

pad_on_right = tokenizer.padding_side == "right"

36 39


In [ ]:
def prepare_train_features(examples):
    """
    Токенизация примеров
    """
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    # Если один пример возращает несколько объектов или если он имеет длинный контекст, то необходима карта от объекта к объекту.
    # Ключ нам возращает карту 
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # Карта смещенных позиций. Используется дальше для вычисление начальных и конечных позиций
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):

        # Невозможные ответы обозначаются индексом CLS
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        #  Последовательность, соответствующую этому примеру
        sequence_ids = tokenized_examples.sequence_ids(i)

        # Один пример может дать несколько промежутков. Это индексы примера, содержащего этот промежуток текста.
        sample_index = sample_mapping[i]
        answers = examples["answer"][sample_index]

        # Если ответов нет, то нашим ответом будет CLS индекс.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Начальный и конечный индекс ответа в тексте
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Начальный и конечный токен в данном нами промежутке
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Если ответ выходит за промежутки, то помечается CLS
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Переместите token_start_index и token_end_index на два конца ответа.
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
# Применяем функцию ко всему датасету

tokenized_datasets = dataset_train.map(prepare_train_features, batched=True, remove_columns=dataset_train["train"].column_names)

### **Загрузка модели**

In [ ]:
# Модель

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [ ]:
# Параметры обучения

args = TrainingArguments(
    f"ntiai",
    evaluation_strategy = "epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1, # 3
    weight_decay=weight_decay,
    save_steps=10000,
    lr_scheduler_type='linear',
)

# Собирает словари

data_collator = default_data_collator

In [ ]:
# Инициализация объекта для обучения

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validate"],
    data_collator=data_collator,  
    tokenizer=tokenizer
)

### ***Обучение***

In [ ]:
trainer.train() 

### **Загрузка теста**

In [ ]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [ ]:
import numpy as np
n_best_size = 5
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: 
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" 
                }
            )

In [ ]:
def prepare_test_features(examples):
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
test_features  = dataset_test["test"].map(
    prepare_test_features,
    batched=True,
    remove_columns=dataset_test["test"].column_names
)

In [ ]:
raw_test = trainer.predict(test_features)

Epoch,Training Loss,Validation Loss


In [ ]:
test_features.set_format(type=test_features.format["type"], columns=list(test_features.features.keys()))

In [ ]:
max_answer_length = 100

In [ ]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()

offset_mapping = test_features[0]["offset_mapping"]

context = dataset_test['test'][0]['context']
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: 
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]

In [ ]:
import collections

examples = dataset_test['test']
features = test_features
example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [ ]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 100):
    all_start_logits, all_end_logits = raw_predictions
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    predictions = collections.OrderedDict()

    for example_index, example in enumerate(tqdm(examples)):
        feature_indices = features_per_example[example_index]

        min_null_score = None 
        valid_answers = []
        
        context = example["context"]
        for feature_index in feature_indices:
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            offset_mapping = features[feature_index]["offset_mapping"]

            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": "", "score": 0.0}
      
        predictions[example["id"]] = best_answer["text"]
        
    return predictions

In [ ]:
final_predictions = postprocess_qa_predictions(dataset_test["test"], test_features, raw_test.predictions)

Post-processing 7258 example predictions split into 7266 features.


In [ ]:
formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]

In [ ]:
formatted_predictions

[{'id': 0, 'prediction_text': ''},
 {'id': 1, 'prediction_text': ' Сергея Скрипаля'},
 {'id': 2, 'prediction_text': ' Ангела Меркель'},
 {'id': 3, 'prediction_text': ' Амир'},
 {'id': 4, 'prediction_text': ' Германия'},
 {'id': 5,
  'prediction_text': ' AFP, что его клиентка находится "на борту самолета", добавив, что пункт назначения воздушного судна неизвестен. Между тем, как передает на своем сайте The News International, крупнейшая англоязычная газета Пакистана, 51-летнюю Биби в полете сопровождал посол Нидерландов'},
 {'id': 6,
  'prediction_text': ' Украине. Антирейтинг возглавили президент Янукович и премьер Азаров. Накануне организация Freedom House назвала украинскую прессу частично свободной. Всеукраинский антирейтинг "Враги прессы" за 2010 год, оглашенный во Всемирный день свободы печати'},
 {'id': 7, 'prediction_text': ' Россию'},
 {'id': 8, 'prediction_text': ' Ганса-Дитриха Геншера'},
 {'id': 9,
  'prediction_text': ' Финляндии, вертолет вторгся вглубь территории страны н

In [ ]:
from difflib import SequenceMatcher

# Сравниваем по схожести предсказанные ответы и ответы, которые даются на выбор
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

js =[]
for i in range(len(test)):
  data = eval(test[i])
  context = data['passage']['text']
  ans_predicted = formatted_predictions[i]['prediction_text']
  ans_predicted = ans_predicted[:3].replace('\n','').replace('\t','').replace(' ','')+ans_predicted[2:]
  c = -10
  for j in data['passage']['entities']:
    ans = context[j['start']:j['end']]
    if similar(ans,ans_predicted) > c:
      c = similar(ans,ans_predicted)
      result_answer = ans
  js.append({'idx':i,'text':result_answer})

In [ ]:
import jsonlines

items = [] 
# сохраняем
with jsonlines.open('arima.jsonl', 'w') as writer:
    writer.write_all(js)